In [1]:
import pandas as pd, numpy as np, seaborn as sns
from pprint import pprint
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import math
from collections import Counter 

import datetime, time

from pprint import pprint
import re

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
def printRuntime():
    print(datetime.datetime.now().strftime("%Y-%m-%d %T"))
    print("-"*19)
    
printRuntime()

2020-02-17 11:23:01
-------------------


# [Goto current work](#current)

# 1. Load data

In [4]:
basePath = "./kalapa/"
trainPath = basePath + "train.csv"
testPath = basePath + "test.csv"
colDesc = basePath + "column_description.csv"
sampleSubmit = basePath + "sample_submission.csv"

cleanedTrain = basePath + "cleanedTrain.pickle"
nb01 = basePath + "nb01.pickle"
nb02 = basePath + "nb02.pickle"
nb03 = basePath + "nb03.pickle"
printRuntime()

2020-02-17 11:23:01
-------------------


In [5]:
def avgAge(row):
    a = row["age_source1"]
    b = row["age_source2"]
    if np.isnan(a) and np.isnan(b):
        return np.nan
    elif np.isnan(a):
        return b
    elif np.isnan(b):
        return a
    else:
        return (a+b)/2.0
    
def diffAge(row):
    a = row["age_source1"]
    b = row["age_source2"]
    if np.isnan(a) and np.isnan(b):
        return np.nan
    elif np.isnan(a):
        return -b
    elif np.isnan(b):
        return a
    else:
        return (a+b)/2.0
    
def groupAge(age):
    # 18-23; 23-35; 35-50; 50+
    result = ""
    if np.isnan(age):
        result = "AGE_NONE"
    elif age <= 18:
        result = "AGE_0"
    elif age <=23:
        result = "AGE_1"
    elif age <=35:
        result = "AGE_2"
    elif age <= 50:
        result = "AGE_3"
    else:
        result = "AGE_4"
    return result
printRuntime()

2020-02-17 11:23:01
-------------------


# Train

In [6]:
pdfTrainRaw = pd.read_csv(trainPath)
printRuntime()

2020-02-17 11:23:02
-------------------


/Users/quyennguyen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
cond = (
    ~((pdfTrainRaw["age_source1"]<18) | (pdfTrainRaw["age_source2"]<18)) # Filter out age < 18, keep NaN
)
pdfTrain = pdfTrainRaw.copy()
# pdfTrain = pdfTrainRaw[cond].copy()
pdfTrain["maCv"] = pdfTrain["maCv"].apply(lambda x: str(x).lower())
pdfTrain["province"] = pdfTrain["province"].apply(lambda x: str(x).lower())
pdfTrain["district"] = pdfTrain["district"].apply(lambda x: str(x).lower())
print(pdfTrain.shape)
printRuntime()

(30000, 64)
2020-02-17 11:23:03
-------------------


# Test

In [8]:
pdfTestRaw = pd.read_csv(testPath)
printRuntime()

2020-02-17 11:23:05
-------------------


/Users/quyennguyen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
cond = (
    ~((pdfTestRaw["age_source1"]<18) | (pdfTestRaw["age_source2"]<18)) # Filter out age < 18, keep NaN
)
pdfTest = pdfTestRaw.copy()
pdfTest["maCv"] = pdfTest["maCv"].apply(lambda x: str(x).lower())
pdfTest["province"] = pdfTest["province"].apply(lambda x: str(x).lower())
pdfTest["district"] = pdfTest["district"].apply(lambda x: str(x).lower())
print(pdfTest.shape)
printRuntime()

(20000, 63)
2020-02-17 11:23:05
-------------------


# Display basic information of FIELDs

In [10]:
# lsMetaCol = ["id", "label", "province", "district", "lv3_loc", 
#               "age_source1", "age_source2", "avg_age", "diff_age", "group_age",
#               "maCv",]

lsMetaCol = ["id", "label", "province", "district",  
              "age_source1", "age_source2", 
              "maCv",]

In [14]:
lsField = ["FIELD_%d"%d for d in range(1, 50)]
for f in lsField:
    print(f)
    lsVal = pdfTrain[f].unique()
    l = len(lsVal)
    print("num of distinct values:", l)
    print("dtypes:", pdfTrain[f].dtypes)
    if l < 20:
        print(lsVal)
    print("-"*20)
printRuntime()

FIELD_1
num of distinct values: 2
dtypes: int64
[1 0]
--------------------
FIELD_2
num of distinct values: 3
dtypes: float64
[ 1.  0. nan]
--------------------
FIELD_3
num of distinct values: 514
dtypes: float64
--------------------
FIELD_4
num of distinct values: 11
dtypes: float64
[ 0.  1. nan  2.  3.  4.  5.  6.  8.  7. 12.]
--------------------
FIELD_5
num of distinct values: 16
dtypes: float64
[ 0.  1.  4.  2. nan  6.  3.  5.  7.  8. 11.  9. 12. 14. 10. 13.]
--------------------
FIELD_6
num of distinct values: 6
dtypes: float64
[ 0.  1. nan  2.  3.  4.]
--------------------
FIELD_7
num of distinct values: 7064
dtypes: object
--------------------
FIELD_8
num of distinct values: 3
dtypes: object
[nan 'MALE' 'FEMALE']
--------------------
FIELD_9
num of distinct values: 35
dtypes: object
--------------------
FIELD_10
num of distinct values: 3
dtypes: object
[nan 'T1' 'GH']
--------------------
FIELD_11
num of distinct values: 43
dtypes: object
--------------------
FIELD_12
num of dis

### Clean data

In [ ]:
25, 29, 30, 31, 36, 37, : 'TRUE' -> True, 'FALSE' -> False !DONE
35: => "Zero"-"Four" -> 1-5 scale (with NULL) !DONE
41: "I"-"V" -> 1-5 scale (with NULL) !DONE
42, 44: "Zezo", "One", "Two" !DONE
45: "1" 1.0 -> 1; "2" 2.0 ->2 !DONE

### Replace "None" as nan, "TRUE"/"FALSE", dupplicated string values DONE!

In [11]:
pdfTrain = (pdfTrain
            .replace(np.nan, "None") #  nan -> 'None', convert back while finish
            .replace("TRUE", True) # 'TRUE' -> True
            .replace("FALSE", False) # 'FALSE' -> False
          )
printRuntime()

2020-02-17 11:23:52
-------------------


In [12]:
pdfTest = (pdfTest
          .replace(np.nan, "None") #  nan -> 'None', convert back while finish
          .replace("TRUE", True) # 'TRUE' -> True
          .replace("FALSE", False) # 'FALSE' -> False
         )
printRuntime()

2020-02-17 11:23:52
-------------------


### Add avg_age, diff_age, group_age DONE!

In [13]:
pdfTrain.loc[pdfTrain["age_source1"]=="None", "age_source1"] = np.nan
pdfTrain.loc[pdfTrain["age_source2"]=="None", "age_source2"] = np.nan

In [14]:
pdfTrain["avg_age"] = pdfTrain.apply(lambda row: avgAge(row), axis=1)
pdfTrain["diff_age"] = pdfTrain.apply(lambda row: diffAge(row), axis=1)
pdfTrain["group_age"] = pdfTrain["avg_age"].apply(lambda x: groupAge(x))
printRuntime()

2020-02-17 11:24:01
-------------------


### Add lv3_loc DONE!

In [15]:
pdfTrain["lv3_loc"] = "[" + pdfTrain["province"] + "]_[" + pdfTrain["district"] + "]"
lsProvince = pdfTrain["province"].unique()
lsLoc = pdfTrain["lv3_loc"].unique()
print(len(lsProvince), len(lsLoc))
printRuntime()

65 766
2020-02-17 11:24:15
-------------------


### Replace ilformed values by columns to safely remove DONE!

In [17]:
pdfTrain["FIELD_6"].head()

0    0
1    1
2    0
3    0
4    0
Name: FIELD_6, dtype: object

In [19]:
# 6
pdfTrain.loc[pdfTrain["FIELD_6"]=="None", "FIELD_6"] = np.nan

In [20]:
# 12
pdfTrain.loc[~pdfTrain["FIELD_12"].isin(["None", "0", "1"]), "FIELD_12"] = "None"

In [21]:
# 35 'Four' 'One' 'Three' 'Two'
pdfTrain.loc[pdfTrain["FIELD_35"]=="Zero", "FIELD_35"] = 1
pdfTrain.loc[pdfTrain["FIELD_35"]=="One", "FIELD_35"] = 2
pdfTrain.loc[pdfTrain["FIELD_35"]=="Two", "FIELD_35"] = 3
pdfTrain.loc[pdfTrain["FIELD_35"]=="Three", "FIELD_35"] = 4
pdfTrain.loc[pdfTrain["FIELD_35"]=="Four", "FIELD_35"] = 5

In [22]:
# 40
pdfTrain.loc[pdfTrain["FIELD_40"].isin(["05 08 11 02", "08 02"]), "FIELD_40"] = "02 05 08 11"
pdfTest.loc[pdfTest["FIELD_40"].isin(["05 08 11 02", "08 02"]), "FIELD_40"] = "02 05 08 11"

In [34]:
ls40Train = pdfTrain["FIELD_40"].unique()

In [35]:
ls40Test = pdfTest["FIELD_40"].unique()

In [36]:
[c for c in ls40Train if c not in ls40Test]

['4']

In [37]:
[c for c in ls40Test if c not in ls40Train]

[nan]

In [23]:
# 41 I II III IV V
pdfTrain.loc[pdfTrain["FIELD_41"]=="I", "FIELD_41"] = 1
pdfTrain.loc[pdfTrain["FIELD_41"]=="II", "FIELD_41"] = 2
pdfTrain.loc[pdfTrain["FIELD_41"]=="III", "FIELD_41"] = 3
pdfTrain.loc[pdfTrain["FIELD_41"]=="IV", "FIELD_41"] = 4
pdfTrain.loc[pdfTrain["FIELD_41"]=="V", "FIELD_41"] = 5

In [24]:
# 42
pdfTrain.loc[pdfTrain["FIELD_42"]=="Zezo", "FIELD_42"] = 0
pdfTrain.loc[pdfTrain["FIELD_42"]=="One", "FIELD_42"] = 1

In [25]:
# 43 0/5 => None
pdfTrain.loc[(pdfTrain["FIELD_43"]=="0")|(pdfTrain["FIELD_43"]=="5"), "FIELD_43"] = np.nan

In [26]:
# 44
pdfTrain.loc[pdfTrain["FIELD_44"]=="One", "FIELD_44"] = 1
pdfTrain.loc[pdfTrain["FIELD_44"]=="Two", "FIELD_44"] = 2

In [27]:
# 45
pdfTrain.loc[(pdfTrain["FIELD_45"]=="1")|(pdfTrain["FIELD_45"]==1.0), "FIELD_45"] = 1
pdfTrain.loc[(pdfTrain["FIELD_45"]=="2")|(pdfTrain["FIELD_45"]==2.0), "FIELD_45"] = 2

### One-hot 8 (gender), 10, 17, 24, 43 DONE!

In [28]:
df_backup = pdfTrain.copy()

In [29]:
d = 17
cName = "FIELD_%d" % d
one_hot = pd.get_dummies(pdfTrain[cName].replace(np.nan, "None"))
print(one_hot)

       G2  G3  G4  G7  G8  G9  GX  None
0       0   0   0   0   0   0   0     1
1       0   0   0   0   1   0   0     0
2       0   0   0   0   0   0   0     1
3       0   0   0   0   0   0   0     1
4       0   0   0   0   1   0   0     0
5       0   0   0   0   0   0   0     1
6       0   0   0   0   0   0   0     1
7       0   0   0   0   0   0   0     1
8       0   0   0   0   0   0   0     1
9       0   0   0   0   0   0   0     1
10      0   0   0   0   0   0   0     1
11      0   0   0   0   0   0   0     1
12      0   0   0   0   0   0   0     1
13      0   0   0   0   0   0   0     1
14      0   0   0   0   0   0   0     1
15      0   0   0   0   1   0   0     0
16      0   0   0   0   0   0   0     1
17      0   0   0   0   0   0   0     1
18      0   0   0   0   0   0   0     1
19      0   0   0   0   0   0   0     1
20      0   0   0   0   0   0   0     1
21      0   0   0   0   1   0   0     0
22      0   0   0   0   0   0   0     1
23      0   0   0   0   0   0   0     1


In [34]:
# Get one hot encoding of columns B
lsCol = [8, 10, 17, 24, 43]
for d in lsCol:
    cName = "FIELD_%d" % d
    one_hot = pd.get_dummies(pdfTrain[cName].replace(np.nan, "None"))
    renameCol = {}
    for c in one_hot.columns:
        renameCol[c] = "%s_%s" % (cName, c)
    pprint(renameCol)
    one_hot = one_hot.rename(columns=renameCol)
    # pdfTrain = pdfTrain.drop(cName, axis = 1)
    # Join the encoded df
    pdfTrain = pdfTrain.join(one_hot)

{'FEMALE': 'FIELD_8_FEMALE', 'MALE': 'FIELD_8_MALE', 'None': 'FIELD_8_None'}
{'GH': 'FIELD_10_GH', 'None': 'FIELD_10_None', 'T1': 'FIELD_10_T1'}
{'G2': 'FIELD_17_G2',
 'G3': 'FIELD_17_G3',
 'G4': 'FIELD_17_G4',
 'G7': 'FIELD_17_G7',
 'G8': 'FIELD_17_G8',
 'G9': 'FIELD_17_G9',
 'GX': 'FIELD_17_GX',
 'None': 'FIELD_17_None'}
{'K1': 'FIELD_24_K1',
 'K2': 'FIELD_24_K2',
 'K3': 'FIELD_24_K3',
 'None': 'FIELD_24_None'}
{'A': 'FIELD_43_A',
 'B': 'FIELD_43_B',
 'C': 'FIELD_43_C',
 'D': 'FIELD_43_D',
 'None': 'FIELD_43_None'}


In [130]:
# Add missing col FIELD_17_G2 = 0 for test
pdfTest["FIELD_17_G2"] = 0

In [35]:
# Count the value for each col 10, 17, 24, 43
pdfTmp = {}
pdfTestTmp = {}
for c in [8, 10, 17, 24, 43]:
    cName = "FIELD_%d" % c
    print("Train")
    pdfTmp[cName] = pdfTrain.groupby([cName], as_index=False).agg({"id": "count"})
    display(pdfTmp[cName])
    print("Test")
    pdfTestTmp[cName] = pdfTest.groupby([cName], as_index=False).agg({"id": "count"})
    display(pdfTestTmp[cName])

Train


,FIELD_8,id
0,FEMALE,9291
1,MALE,11031
2,None,9678


Test


,FIELD_8,id
0,FEMALE,6086
1,MALE,7410
2,None,6504


Train


...

,FIELD_24,id
0,K1,114
1,K2,154
2,K3,60
3,None,19672


Train


,FIELD_43,id
0,A,114
1,B,373
2,C,630
3,D,65
4,None,28804


Test


,FIELD_43,id
0,0,1
1,5,5
2,A,68
3,B,248
4,C,437
5,D,30
6,None,19211


### Target/Mean-Encoding for 8 (gender), 10, 17, 24, 43

In [36]:
df_backup = pdfTrain.copy()

In [78]:
# Reload pdfTrain
pdfTrain = df_backup.copy()

In [77]:
d = 17
cName = "FIELD_%d" % d
pdf = pdfTrain
meanEcd = pdf.groupby([cName], as_index=False).agg({"label":["mean", "std"]})
meanEcd.columns = ["_".join(x) for x in meanEcd.columns.ravel()]
meanEcd = meanEcd.rename(columns={cName+"_": cName, 
                                  "label_mean": cName+"_mean", 
                                  "label_std": cName+"_std"})
# display(meanEcd)
pdfTrain = pd.merge(pdfTrain, meanEcd, on=cName, how="inner")
display(pdfTrain[[cName, "label", "id", cName+"_mean", cName+"_std"]].iloc[:5])
printRuntime()

,FIELD_17,label,id,FIELD_17_mean,FIELD_17_std
0,None,0,0,0.015052,0.121762
1,None,0,2,0.015052,0.121762
2,None,0,3,0.015052,0.121762
3,None,0,5,0.015052,0.121762
4,None,0,6,0.015052,0.121762


2020-02-17 11:54:38
-------------------


In [72]:
pdfTrain.columns

Index(['id', 'label', 'province', 'district', 'age_source1', 'age_source2',
       'maCv', 'FIELD_1', 'FIELD_2', 'FIELD_3', 'FIELD_4', 'FIELD_5',
       'FIELD_6', 'FIELD_7', 'FIELD_8', 'FIELD_9', 'FIELD_10', 'FIELD_11',
       'FIELD_12', 'FIELD_13', 'FIELD_14', 'FIELD_15', 'FIELD_16', 'FIELD_17',
       'FIELD_18', 'FIELD_19', 'FIELD_20', 'FIELD_21', 'FIELD_22', 'FIELD_23',
       'FIELD_24', 'FIELD_25', 'FIELD_26', 'FIELD_27', 'FIELD_28', 'FIELD_29',
       'FIELD_30', 'FIELD_31', 'FIELD_32', 'FIELD_33', 'FIELD_34', 'FIELD_35',
       'FIELD_36', 'FIELD_37', 'FIELD_38', 'FIELD_39', 'FIELD_40', 'FIELD_41',
       'FIELD_42', 'FIELD_43', 'FIELD_44', 'FIELD_45', 'FIELD_46', 'FIELD_47',
       'FIELD_48', 'FIELD_49', 'FIELD_50', 'FIELD_51', 'FIELD_52', 'FIELD_53',
       'FIELD_54', 'FIELD_55', 'FIELD_56', 'FIELD_57', 'avg_age', 'diff_age',
       'group_age', 'lv3_loc', 'FIELD_8_FEMALE', 'FIELD_8_MALE',
       'FIELD_8_None', 'FIELD_10_GH', 'FIELD_10_None', 'FIELD_10_T1',
       'FIELD

In [74]:
pdfTrain = pd.merge(pdfTrain, meanEcd, on=cName, how="inner")
printRuntime()

2020-02-17 11:53:12
-------------------


In [75]:
pdfTrain[[cName, "label", "id", cName+"_mean", cName+"_std"]].head()

,FIELD_17,label,id,FIELD_17_mean,FIELD_17_std
0,None,0,0,0.015052,0.121762
1,None,0,2,0.015052,0.121762
2,None,0,3,0.015052,0.121762
3,None,0,5,0.015052,0.121762
4,None,0,6,0.015052,0.121762


In [79]:
# Get target/mean encoding of columns B
lsCol = [8, 10, 17, 24, 43]
for d in lsCol:
    cName = "FIELD_%d" % d
    meanEcd = pdfTrain.groupby([cName], as_index=False).agg({"label":["mean", "std"]})
    meanEcd.columns = ["_".join(x) for x in meanEcd.columns.ravel()]
    meanEcd = meanEcd.rename(columns={cName+"_": cName, 
                                      "label_mean": cName+"_ecdmean", 
                                      "label_std": cName+"_ecdstd"})
    # display(meanEcd)
    pdfTrain = pd.merge(pdfTrain, meanEcd, on=cName, how="inner")
    display(pdfTrain[[cName, "label", "id", cName+"_mean", cName+"_std"]].iloc[:5])
    printRuntime()

,FIELD_8,label,id,FIELD_8_mean,FIELD_8_std
0,None,0,0,0.020872,0.142963
1,None,0,3,0.020872,0.142963
2,None,0,5,0.020872,0.142963
3,None,0,8,0.020872,0.142963
4,None,0,9,0.020872,0.142963


2020-02-17 11:55:19
-------------------


,FIELD_10,label,id,FIELD_10_mean,FIELD_10_std
0,None,0,0,0.020872,0.142963
1,None,0,3,0.020872,0.142963
2,None,0,5,0.020872,0.142963
3,None,0,8,0.020872,0.142963
4,None,0,9,0.020872,0.142963


2020-02-17 11:55:19
-------------------


,FIELD_17,label,id,FIELD_17_mean,FIELD_17_std
0,None,0,0,0.015052,0.121762
1,None,0,3,0.015052,0.121762
2,None,0,5,0.015052,0.121762
3,None,0,8,0.015052,0.121762
4,None,0,9,0.015052,0.121762


2020-02-17 11:55:20
-------------------


,FIELD_24,label,id,FIELD_24_mean,FIELD_24_std
0,None,0,0,0.016203,0.126257
1,None,0,3,0.016203,0.126257
2,None,0,5,0.016203,0.126257
3,None,0,8,0.016203,0.126257
4,None,0,9,0.016203,0.126257


2020-02-17 11:55:20
-------------------


,FIELD_43,label,id,FIELD_43_mean,FIELD_43_std
0,None,0,0,0.016456,0.127223
1,None,0,3,0.016456,0.127223
2,None,0,5,0.016456,0.127223
3,None,0,8,0.016456,0.127223
4,None,0,9,0.016456,0.127223


2020-02-17 11:55:20
-------------------


In [ ]:
# Count the value for each col 10, 17, 24, 43
pdfTmp = {}
pdfTestTmp = {}
for c in [8, 10, 17, 24, 43]:
    cName = "FIELD_%d" % c
    print("Train")
    pdfTmp[cName] = (pdfTrain.groupby([cName], as_index=False)
                     .agg({"id": "count", "label":["mean", "std"]})
                    )
    display(pdfTmp[cName])
    print("Test")
    pdfTestTmp[cName] = (pdfTest.groupby([cName], as_index=False)
                         .agg({"id": "count", "label":["mean", "std"]})
                        )
    display(pdfTestTmp[cName])

In [31]:
# Data to get more information: 3, 4, 5, 11, 13, 39, 40
lsObjField = [3, 4, 5, 7, 9, 11, 13, 39, 40]
for d in lsObjField:
    col = "FIELD_%d"%d
    print(col)
    lsUniqueVal = pdfTrain[col].unique()
    print(len(lsUniqueVal))
    print(lsUniqueVal[:50])
    print("-"*20)
printRuntime()

FIELD_3
514
[ 2.547e+03 -1.000e+00  3.273e+03  3.991e+03  1.450e+03  1.812e+03
  2.906e+03        nan  2.544e+03  2.168e+03  3.262e+03  7.060e+02
  1.436e+03  7.210e+02  3.550e+02  3.260e+03  3.637e+03  3.390e+02
  3.500e+02  3.650e+02  3.267e+03  3.540e+02  1.071e+03  3.630e+02
  2.174e+03  1.826e+03  4.009e+03  1.084e+03  2.186e+03  1.444e+03
  3.992e+03  3.420e+02  3.989e+03  3.670e+02  1.824e+03  2.923e+03
  2.897e+03  7.250e+02  2.548e+03  2.924e+03  4.012e+03  3.651e+03
  2.541e+03  1.438e+03  4.007e+03  2.915e+03  3.631e+03  3.284e+03
  2.189e+03  1.440e+03]
--------------------
FIELD_4
11
[ 0.  1. nan  2.  3.  4.  5.  6.  8.  7. 12.]
--------------------
FIELD_5
16
[ 0.  1.  4.  2. nan  6.  3.  5.  7.  8. 11.  9. 12. 14. 10. 13.]
--------------------
FIELD_7
7064
['[]' "['GD', 'GD', 'TE']" "['DN', 'HN', 'DN']" "['CH', 'TQ']"
 "['SV', 'GD']" "['HS', 'GB', 'DN', 'DN']" nan "['DN', 'DN', 'GD']"
 "['HT', 'DN', 'TN', 'GD', 'GD', 'TN', 'GD']" "['DN']" "['DN', 'DK']"
 "['XD', 'XD', 'X

In [123]:
lsField = [c for c in df.columns if c not in ["id", "label"]]
# ["FIELD_%d"%d for d in range(1, 50)]
for f in lsField:
    print(f)
    lsVal = df[f].unique()
    l = len(lsVal)
    print(l, "dtypes:", df[f].dtypes)
    if l < 10:
        print(lsVal)
    print("-"*20)
printRuntime()

province
66 dtypes: object
--------------------
district
719 dtypes: object
--------------------
age_source1
51 dtypes: float64
--------------------
age_source2
61 dtypes: float64
--------------------
maCv
3064 dtypes: object
--------------------
FIELD_1
2 dtypes: int64
[1 0]
--------------------
FIELD_2
3 dtypes: float64
[ 1.  0. nan]
--------------------
FIELD_3
514 dtypes: float64
--------------------
FIELD_4
11 dtypes: float64
--------------------
FIELD_5
16 dtypes: float64
--------------------
FIELD_6
6 dtypes: float64
[ 1.  2. nan  3.  4.  5.]
--------------------
FIELD_7
7064 dtypes: object
--------------------
FIELD_8
3 dtypes: object
[nan 'MALE' 'FEMALE']
--------------------
FIELD_9
35 dtypes: object
--------------------
FIELD_11
42 dtypes: object
--------------------
FIELD_13
227 dtypes: object
--------------------
FIELD_14
2 dtypes: int64
[1 0]
--------------------
FIELD_15
2 dtypes: int64
[1 0]
--------------------
FIELD_16
6 dtypes: float64
[nan  2.  1.  3.  4.  5.]
-----

### Check FIELD_7 and FIELD_9

In [20]:
pdfTrain[lsMetaCol + ["FIELD_7", "FIELD_9"]].head(20)

,id,label,province,district,age_source1,age_source2,maCv,FIELD_7,FIELD_9
0,0,0,NaN,NaN,NaN,NaN,NaN,[],na
1,1,0,Tỉnh Đồng Nai,Thành phố Biên Hòa,44.0,44.0,None,"['GD', 'GD', 'TE']",GD
2,2,0,Tỉnh Đồng Nai,Huyện Long Thành,30.0,30.0,Công nhân,"['DN', 'HN', 'DN']",DN
3,3,0,Tỉnh Tuyên Quang,Thành phố Tuyên Quang,43.0,NaN,NaN,"['CH', 'TQ']",na
4,4,0,Thành phố Hồ Chí Minh,Quận 1,21.0,21.0,None,"['SV', 'GD']",GD
5,5,0,NaN,NaN,NaN,NaN,NaN,[],na
6,6,0,NaN,NaN,NaN,28.0,2983,[],DN
7,7,0,Tỉnh Bắc Giang,Huyện Yên Dũng,40.0,32.0,Công nhân,"['HS', 'GB', 'DN', 'DN']",DN
8,8,0,NaN,NaN,NaN,NaN,NaN,NaN,na
9,9,0,NaN,NaN,NaN,NaN,NaN,[],na


In [21]:
pdfTrain[lsMetaCol + ["FIELD_7", "FIELD_9"]].loc[pdfTrain["label"]==1].head(20)

,id,label,province,district,age_source1,age_source2,maCv,FIELD_7,FIELD_9
106,106,1,Thành phố Hồ Chí Minh,Quận 6,32.0,NaN,NaN,"['GD', 'TN']",na
113,113,1,NaN,NaN,NaN,NaN,NaN,[],na
157,157,1,Tỉnh Kiên Giang,Huyện Giồng Riềng,40.0,40.0,CBKCT,"['GD', 'GD', 'XK', 'GD']",GD
163,163,1,NaN,NaN,NaN,28.0,None,[],DN
296,296,1,Thành phố Hà Nội,Huyện Phú Xuyên,35.0,NaN,NaN,"['HS', 'TE', 'HT', 'HT', 'HC']",na
377,377,1,Tỉnh Vĩnh Long,Thành phố Vĩnh Long,29.0,29.0,Giáo viên Trường Mầm non 5 phường 5,"['TE', 'HC', 'BT']",HC
384,384,1,NaN,NaN,NaN,25.0,None,[],DN
580,580,1,NaN,NaN,NaN,25.0,None,[],DN
746,746,1,Tỉnh Đồng Nai,Huyện Long Thành,48.0,48.0,Nhân viên Phụ bếp,"['DN', 'DN', 'DN']",TN
816,816,1,NaN,NaN,NaN,NaN,NaN,[],na


In [71]:
ls7Val = pdfTrain["FIELD_7"].unique()
print(len(ls7Val))

7064


In [74]:
# Break down FIELD_7 value
tmp = []
for c in ls7Val:
    if c == "[]" or c is np.nan:
        tmp.append("na")
        continue
    s = c.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    # print(c, s)
    tmp.extend(s)
ls7UniqueVal = list(set(tmp)) # => Vocab of FIELD_7
print(ls7UniqueVal)
print(len(ls7UniqueVal))

['HT', 'TN', 'CC', 'TS', 'na', 'MS', 'CN', 'KC', 'GD', 'HX', 'HD', 'CB', 'CK', 'HS', 'SV', 'NO', 'TK', 'PV', 'TB', 'XK', 'XV', 'LS', 'TA', 'DN', 'TQ', 'DT', 'BT', 'QN', 'TE', 'DK', 'TC', 'HC', 'CH', 'XN', 'XD', 'HN', 'NN', 'GB']
38


In [16]:
def splitF7(f7):
    if f7 == "[]" or f7 is np.nan:
        return "na"
    s = f7.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    return s

In [130]:
pdfTrain["FIELD_7_VECTOR"] = pdfTrain["FIELD_7"].apply(splitF7)

In [131]:
pdfTrain[["FIELD_7", "FIELD_7_VECTOR"]].head()

,FIELD_7,FIELD_7_VECTOR
0,[],na
1,"['GD', 'GD', 'TE']","[GD, GD, TE]"
2,"['DN', 'HN', 'DN']","[DN, HN, DN]"
3,"['CH', 'TQ']","[CH, TQ]"
4,"['SV', 'GD']","[SV, GD]"


In [75]:
# FIELD_9 is not fully covered by FIELD_7
ls9Val = pdfTrain["FIELD_9"].unique()

print(ls9Val)
print(len(ls9Val))

['na' 'GD' 'DN' 'XD' 'HC' 'TN' 'CH' 'CN' 'HT' 'DT' 'XK' 'TK' 'GB' 'DK'
 'SV' 'HN' 'TS' 'TA' 'HD' 'NN' 'BT' 'HS' 'HX' 'NO' 'KC' 'CB' 'TC' 'XV'
 'XN' 'CC' 'MS']
31


In [76]:
lsDiff1 = [c for c in ls7UniqueVal if c not in ls9Val] 
print(lsDiff1) # in 7 not in 9
lsDiff2 = [c for c in ls9Val if c not in ls7UniqueVal]
print(lsDiff2) # in 9 not in 7

['CK', 'PV', 'TB', 'LS', 'TQ', 'QN', 'TE']
[]


In [17]:
def mostFrequent(List): 
    occurenceCount = Counter(List) 
    lsCount = occurenceCount.most_common()
    maxAppear = lsCount[0][1]
    # print(lsCount)
    # print(maxAppear)
    lsVal = [v[0] for v in lsCount if v[1] == maxAppear]
    # Return list of most common items (if equally appeared)
    return lsVal

def mostAppear(iStr):
    if iStr == "[]" or iStr is np.nan:
        return "na"
    lsChar = iStr.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    return mostFrequent(lsChar)

# Test
mostFrequent([2, 2, 3, 3, 3, 4, 5, 6]) # Expect [2, 3]
print(mostAppear("['GD', 'GD', 'TE']")) # Expect ["GD"]

def index_7_9(row):
    """
    Return: (is_appear, index)
    """
    lsMostAppear7 = row["FIELD_7_MOST"]
    val9 = row["FIELD_9"]
    if val9 == "[]" or val9 == "na":
        return -2, -1
    elif lsMostAppear7 == "na":
        return -1, -1
    elif val9 in lsMostAppear7:
        return 1, lsMostAppear7.index(val9)
    else:
        # lsMostAppear7 is not null but 9 doesn't appear
        return 0, -1
    
printRuntime()

2020-02-12 00:15:43
-------------------


In [80]:
pdfTrain[lsMetaCol + ["FIELD_7", "FIELD_7_MOST", "FIELD_9"]].head(20)

,id,label,province,district,age_source1,age_source2,maCv,FIELD_7,FIELD_7_MOST,FIELD_9
0,0,0,nan,NaN,NaN,NaN,nan,[],na,na
1,1,0,tỉnh đồng nai,Thành phố Biên Hòa,44.0,44.0,none,"['GD', 'GD', 'TE']",[GD],GD
2,2,0,tỉnh đồng nai,Huyện Long Thành,30.0,30.0,công nhân,"['DN', 'HN', 'DN']",[DN],DN
3,3,0,tỉnh tuyên quang,Thành phố Tuyên Quang,43.0,NaN,nan,"['CH', 'TQ']","[CH, TQ]",na
4,4,0,thành phố hồ chí minh,Quận 1,21.0,21.0,none,"['SV', 'GD']","[SV, GD]",GD
5,5,0,nan,NaN,NaN,NaN,nan,[],na,na
6,6,0,nan,NaN,NaN,28.0,2983,[],na,DN
7,7,0,tỉnh bắc giang,Huyện Yên Dũng,40.0,32.0,công nhân,"['HS', 'GB', 'DN', 'DN']",[DN],DN
8,8,0,nan,NaN,NaN,NaN,nan,NaN,na,na
9,9,0,nan,NaN,NaN,NaN,nan,[],na,na


In [92]:
pdfTrain["F7_MOST_IDX"] = pdfTrain["FIELD_7_MOST"].astype(str)

In [94]:
pdfTrain["F7_LEN"] = pdfTrain["FIELD_7_MOST"].apply(len)

In [ ]:
pdfTmp = pdfTrain.groupby(["F7_LEN"], as_index=False).agg({"id":"count"})

In [96]:
pdfTmp.head(10)

,F7_LEN,id
0,1,12115
1,2,16033
2,3,1165
3,4,482
4,5,107
5,6,21
6,7,1


In [ ]:
# Add col as most appear FIELD_7
pdfTrain["FIELD_7_MOST"] = pdfTrain["FIELD_7"].apply(lambda x: mostAppear(x))
pdfTrain["FIELD_7_LEN"] = pdfTrain["FIELD_7_MOST"].apply(len)
# Add col as 9 appear in 7_MOST
pdfTrain["FIELD_9_APPEAR"], pdfTrain["FIELD_9_INDEX"] = zip(*pdfTrain.apply(lambda row: index_7_9(row), axis=1))

In [98]:
pdfTrain[lsMetaCol + ["FIELD_7", "FIELD_7_MOST", "FIELD_9", "FIELD_9_APPEAR", "FIELD_9_INDEX",]].head(20)

In [99]:
pdfTmp = pdfTrain.groupby(["FIELD_9_APPEAR"], as_index=False).agg({"id":"count",})
pdfTmp.head(5)

,FIELD_9_APPEAR,id
0,-2,9684
1,-1,4936
2,0,5061
3,1,10243


In [100]:
pdfTmpIdx = pdfTrain.groupby(["FIELD_9_INDEX"], as_index=False).agg({"id":"count",})
pdfTmpIdx.head(10)

,FIELD_9_INDEX,id
0,-1,19681
1,0,8172
2,1,1509
3,2,425
4,3,111
5,4,23
6,5,3


In [53]:
pdfTrain[lsMetaCol + ["FIELD_7", "FIELD_7_MOST", "FIELD_9", "FIELD_9_APPEAR",]
        ].loc[pdfTrain["FIELD_9_APPEAR"]==3].head(20)

,id,label,province,district,age_source1,age_source2,maCv,FIELD_7,FIELD_7_MOST,FIELD_9,FIELD_9_APPEAR
166,166,0,Tỉnh Nghệ An,Thị xã Thái Hoà,28.0,28.0,kế toán,"['TE', 'GD', 'DN', 'TK']","[TE, GD, DN, TK]",TK,3
302,302,0,Tỉnh Long An,Huyện Cần Đước,41.0,41.0,công nhân,"['HS', 'BT', 'GD', 'DN']","[HS, BT, GD, DN]",DN,3
390,390,0,Tỉnh Quảng Nam,Huyện Duy Xuyên,33.0,33.0,nhân viên,"['TE', 'DK', 'HS', 'DN']","[TE, DK, HS, DN]",DN,3
566,566,0,Tỉnh Vĩnh Phúc,Huyện Bình Xuyên,32.0,32.0,nhân viên phòng thí nghiệm,"['TN', 'HT', 'TE', 'DN']","[TN, HT, TE, DN]",DN,3
674,674,0,Tỉnh Khánh Hòa,Thị xã Ninh Hòa,26.0,26.0,công nhân ép keo,"['GD', 'SV', 'TE', 'CH', 'HD']","[GD, SV, TE, CH, HD]",CH,3
1006,1006,0,Tỉnh Yên Bái,Huyện Yên Bình,34.0,34.0,none,"['TE', 'HS', 'HC', 'DN']","[TE, HS, HC, DN]",DN,3
1194,1194,0,Tỉnh Bình Phước,Huyện Lộc Ninh,31.0,31.0,none,"['DK', 'CK', 'TE', 'DN']","[DK, CK, TE, DN]",DN,3
1561,1561,0,Tỉnh Bến Tre,Thành phố Bến Tre,27.0,27.0,none,"['GD', 'HS', 'TE', 'HC', 'KC']","[GD, HS, TE, HC, KC]",HC,3
1653,1653,0,Thành phố Hồ Chí Minh,Quận Thủ Đức,22.0,22.0,none,"['HS', 'SV', 'TE', 'DN']","[HS, SV, TE, DN]",DN,3
1693,1693,0,Tỉnh Bình Định,Thành phố Qui Nhơn,53.0,53.0,nhân viên hành chính kiêm lái xe,"['SV', 'DN', 'HS', 'CH']","[SV, DN, HS, CH]",CH,3


### Check Test

In [101]:
ls7TestVal = pdfTest["FIELD_7"].unique()
print(len(ls7TestVal))

4905


In [102]:
# Break down FIELD_7 value
tmp = []
for c in ls7TestVal:
    if c == "[]" or c is np.nan:
        tmp.append("na")
        continue
    s = c.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    # print(c, s)
    tmp.extend(s)
ls7TestUniqueVal = list(set(tmp)) # => Vocab of FIELD_7
print(ls7TestUniqueVal)
print(len(ls7TestUniqueVal))

['HT', 'TN', 'TS', 'CC', 'na', 'MS', 'CN', 'HK', 'KC', 'GD', 'HX', 'HD', 'HG', 'CB', 'CK', 'HS', 'SV', 'NO', 'AT', 'TK', 'PV', 'ND', 'TB', 'XK', 'XV', 'TA', 'DN', 'TQ', 'QT', 'DT', 'BT', 'TE', 'XB', 'DK', 'TC', 'HC', 'CH', 'XN', 'XD', 'TL', 'HN', 'NN', 'GB']
43


In [103]:
# FIELD_9 is not fully covered by FIELD_7
ls9TestVal = pdfTest["FIELD_9"].unique()

print(ls9TestVal)
print(len(ls9TestVal))

['na' 'DN' 'GD' 'DK' 'DT' 'BT' 'TN' 'CN' 'CH' 'HC' 'GB' 'HN' 'HT' 'SV'
 'TA' 'XK' 'XD' 'TK' 'CB' 'NN' 'TS' 'KC' 'XV' 'HD' 'HX' 'TC' 'HS' '75'
 '74' 'TL' 'CC' 'CK']
32


In [123]:
# Add col as most appear FIELD_7
pdfTest["FIELD_7_MOST"] = pdfTest["FIELD_7"].apply(lambda x: mostAppear(x))

In [124]:
pdfTest["F7_LEN"] = pdfTest["FIELD_7_MOST"].apply(len)

In [125]:
# Add col as 9 appear in 7_MOST
pdfTest["FIELD_9_APPEAR"], pdfTest["FIELD_9_INDEX"] = zip(*pdfTest.apply(lambda row: index_7_9(row), axis=1))

In [126]:
pdfTest.loc[pdfTest["FIELD_9"].isin(["CC", "CK", "TC", "TL", "HD", "XV"])][
    ["FIELD_7", "FIELD_7_MOST", "FIELD_9", "FIELD_9_APPEAR", "FIELD_9_INDEX"]].head(30)

,FIELD_7,FIELD_7_MOST,FIELD_9,FIELD_9_APPEAR,FIELD_9_INDEX
1791,[],na,XV,-1,-1
2033,[],na,HD,-1,-1
2524,[],na,TC,-1,-1
4155,[],na,XV,-1,-1
5511,"['HS', 'HD']","[HS, HD]",HD,1,1
5978,"['XD', 'XD', 'XD', 'XD', 'XD']",[XD],XV,0,-1
7213,"['HD', 'GB', 'GB', 'BT']",[GB],HD,0,-1
10903,[],na,TL,-1,-1
11410,[],na,HD,-1,-1
11817,"['GD', 'DN', 'TE', 'GD', 'GD']",[GD],XV,0,-1


In [117]:
dfTmp = pdfTest.groupby("FIELD_9").agg({"id": "count"})
display(dfTmp)

,id
FIELD_9,
74,3
75,3
BT,14
CB,14
CC,1
CH,383
CK,1
CN,94
DK,177


In [116]:
dfTmp = pdfTest.loc[pdfTest["FIELD_9"].isin(["74", "75"])].groupby("FIELD_9").agg({"id": "count"})
display(dfTmp)

,id
FIELD_9,
74,3
75,3


In [104]:
lsDiff1 = [c for c in ls7TestUniqueVal if c not in ls9TestVal] 
print(lsDiff1) # in 7 not in 9
lsDiff2 = [c for c in ls9TestVal if c not in ls7TestUniqueVal]
print(lsDiff2) # in 9 not in 7

['MS', 'HK', 'HG', 'NO', 'AT', 'PV', 'ND', 'TB', 'TQ', 'QT', 'TE', 'XB', 'XN']
['75', '74']


In [105]:
print("in 7train but not in 7test")
ls7TrainOnly = [c for c in ls7UniqueVal if c not in ls7TestUniqueVal] 
print(ls7TrainOnly)

in 7train but not in 7test
['LS', 'QN']


In [106]:
print("in 7test but not in 7train")
ls7TestOnly = [c for c in ls7TestUniqueVal if c not in ls7UniqueVal] 
print(ls7TestOnly)

in 7test but not in 7train
['HK', 'HG', 'AT', 'ND', 'QT', 'XB', 'TL']


In [105]:
print("in 7train but not in 7test")
ls7TrainOnly = [c for c in ls7UniqueVal if c not in ls7TestUniqueVal] 
print(ls7TrainOnly)

in 7train but not in 7test
['LS', 'QN']


In [106]:
print("in 7test but not in 7train")
ls7TestOnly = [c for c in ls7TestUniqueVal if c not in ls7UniqueVal] 
print(ls7TestOnly)

in 7test but not in 7train
['HK', 'HG', 'AT', 'ND', 'QT', 'XB', 'TL']


# TODO: With FIELD_7 and FIELD_9
    Combine vocabulary of (7, 9) * (train, test)
    One-hot and sum the vector for 7, add normalized for 7
    One-hot for 9
    Also keep FIELD_7_MOST -> one-hot, FIELD_7_LEN, FIELD_9_APPEAR, FIELD_9_INDEX as features

In [113]:
ls7UniqueVal

['HT',
 'TN',
 'CC',
 'TS',
 'na',
 'MS',
 'CN',
 'KC',
 'GD',
 'HX',
 'HD',
 'CB',
 'CK',
 'HS',
 'SV',
 'NO',
 'TK',
 'PV',
 'TB',
 'XK',
 'XV',
 'LS',
 'TA',
 'DN',
 'TQ',
 'DT',
 'BT',
 'QN',
 'TE',
 'DK',
 'TC',
 'HC',
 'CH',
 'XN',
 'XD',
 'HN',
 'NN',
 'GB',
 'HT',
 'TN',
 'TS',
 'CC',
 'na',
 'MS',
 'CN',
 'HK',
 'KC',
 'GD',
 'HX',
 'HD',
 'HG',
 'CB',
 'CK',
 'HS',
 'SV',
 'NO',
 'AT',
 'TK',
 'PV',
 'ND',
 'TB',
 'XK',
 'XV',
 'TA',
 'DN',
 'TQ',
 'QT',
 'DT',
 'BT',
 'TE',
 'XB',
 'DK',
 'TC',
 'HC',
 'CH',
 'XN',
 'XD',
 'TL',
 'HN',
 'NN',
 'GB']

In [111]:
ls9TestVal

array(['na', 'DN', 'GD', 'DK', 'DT', 'BT', 'TN', 'CN', 'CH', 'HC', 'GB',
       'HN', 'HT', 'SV', 'TA', 'XK', 'XD', 'TK', 'CB', 'NN', 'TS', 'KC',
       'XV', 'HD', 'HX', 'TC', 'HS', '75', '74', 'TL', 'CC', 'CK'],
      dtype=object)

In [110]:
ls9Val

array(['na', 'GD', 'DN', 'XD', 'HC', 'TN', 'CH', 'CN', 'HT', 'DT', 'XK',
       'TK', 'GB', 'DK', 'SV', 'HN', 'TS', 'TA', 'HD', 'NN', 'BT', 'HS',
       'HX', 'NO', 'KC', 'CB', 'TC', 'XV', 'XN', 'CC', 'MS'], dtype=object)

In [114]:
set(ls7UniqueVal + ls7TestUniqueVal + list(ls9Val) + list(ls9TestVal))

{'74',
 '75',
 'AT',
 'BT',
 'CB',
 'CC',
 'CH',
 'CK',
 'CN',
 'DK',
 'DN',
 'DT',
 'GB',
 'GD',
 'HC',
 'HD',
 'HG',
 'HK',
 'HN',
 'HS',
 'HT',
 'HX',
 'KC',
 'LS',
 'MS',
 'ND',
 'NN',
 'NO',
 'PV',
 'QN',
 'QT',
 'SV',
 'TA',
 'TB',
 'TC',
 'TE',
 'TK',
 'TL',
 'TN',
 'TQ',
 'TS',
 'XB',
 'XD',
 'XK',
 'XN',
 'XV',
 'na'}

In [115]:
lsVocab = list(set(ls7UniqueVal + ls7TestUniqueVal + list(ls9Val) + list(ls9TestVal)))
print(len(lsVocab))

47


# TODO:
    - Do word/doc embedding with field 7, 9, 13, 39
    - Separate train/test
    - Build the vocabulary on train only. New values on test will be assigned as 'na'.
    - 1 input layer + 1 hidden layer + 1 softmax layer output 0/1
    - F7: DOC2VEC
    - F9, F13, F39: WORD2VEC

In [142]:
def splitF7(f7):
    if f7 == "[]" or f7 is np.nan:
        return ["na"]
    s = f7.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    return s

def toOneHot(dataPointIndex, vocabSize):
    temp = np.zeros(vocabSize)
    temp[dataPointIndex] = 1
    return temp

def encodeOneHotVector(row):
    temp = np.zeros(vocabSize)
    for c in row:
        temp[word2int[c]] = temp[word2int[c]] + 1
    return temp

printRuntime()

2020-02-10 01:47:44
-------------------


In [136]:
# Separate field 7 into vector
pdfTrain["FIELD_7_VECTOR"] = pdfTrain["FIELD_7"].apply(splitF7)

In [137]:
# Build vocabulary
vocab = []
for r in pdfTrain["FIELD_7_VECTOR"]:
    vocab.extend(r)
vocab = list(set(vocab))
printRuntime()

2020-02-10 01:35:12
-------------------


In [140]:
# Build 1-hot lookup table
word2int = {}
int2word = {}
vocabSize = len(vocab) # gives the total number of unique words
for i,word in enumerate(vocab):
    word2int[word] = i
    int2word[i] = word

In [143]:
# Build input layer as sum of each word in a vector
pdfTrain["FIELD_7_ONEHOT"] = pdfTrain["FIELD_7_VECTOR"].apply(encodeOneHotVector)

In [150]:
pdfTrain["F7_SUM"] = pdfTrain["FIELD_7_ONEHOT"].apply(sum)
pdfTrain["F7_CHECKSUM"] = pdfTrain["FIELD_7_VECTOR"].apply(len)

In [151]:
pdfTrain[["FIELD_7", "FIELD_7_VECTOR", "F7_CHECKSUM", "FIELD_7_ONEHOT", "F7_SUM"]].head()

,FIELD_7,FIELD_7_VECTOR,F7_CHECKSUM,FIELD_7_ONEHOT,F7_SUM
0,[],[na],1,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
1,"['GD', 'GD', 'TE']","[GD, GD, TE]",3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.0
2,"['DN', 'HN', 'DN']","[DN, HN, DN]",3,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",3.0
3,"['CH', 'TQ']","[CH, TQ]",2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0
4,"['SV', 'GD']","[SV, GD]",2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0


In [20]:
# Build a neural network with 1 input layer (1-hot); 
# 1 hidden layer (hidden_size configurable); 1 softmax layer
EMBEDDING_SIZE = 10

# Train on label

# Output from hidden layer will be used as encoding

In [21]:
def encodeOneHot(row, vocabSize, word2int):
    dataPointIndex = word2int[row]
    temp = np.zeros(vocabSize)
    temp[dataPointIndex] = 1
    return temp

def encodeOneHotVector(row, vocabSize, word2int):
    temp = np.zeros(vocabSize)
    for c in row:
        temp[word2int[c]] = temp[word2int[c]] + 1
    return temp

def encodeCol(pdfTrain, iCol):
    """
    Args:
        dfTrain: train part, only encode column based on this dataframe
        col: column name to be encoded
    Returns:
        vocab: Vocabulary -> encode
        word2int: lookup table word2int -> encode
        int2word: lookup table int2word -> encode
        pdfTrainEncode: pdf with encoded column (1-hot/1-hot sum vector)
    """
    vocab = []
    word2int = {}
    int2word = {}
    # Separate field 7 into vector
    if "FIELD_7" == iCol:
        # Special treatment for FIELD_7
        pdfTrain["FIELD_7_VECTOR"] = pdfTrain["FIELD_7"].apply(splitF7)
        # Build vocabulary
        for r in pdfTrain["FIELD_7_VECTOR"]:
            vocab.extend(r)
        vocab = list(set(vocab))
    else:
        # Build vocabulary
        pdfTrain.loc[pdfTrain[iCol].isnull(), iCol] = "None" # Prevent nan value for key
        vocab = list(pdfTrain[iCol].unique())
    # Build 1-hot lookup table
    vocabSize = len(vocab) # gives the total number of unique words
    print(vocab)
    print(vocabSize)
    for i,word in enumerate(vocab):
        word2int[word] = i
        int2word[i] = word
    # apply 1-hot
    if "FIELD_7" == iCol:
        newColSeries = pdfTrain["FIELD_7_VECTOR"].apply(lambda r: encodeOneHotVector(r, vocabSize, word2int))
    else:
        newColSeries = pdfTrain[iCol].apply(lambda r: encodeOneHot(r, vocabSize, word2int))
    # Return
    return vocab, word2int, int2word, newColSeries

printRuntime()

2020-02-12 00:16:10
-------------------


In [22]:
vocabF7, word2intF7, int2wordF7, pdfTrain["FIELD_7_ONEHOT_TEST"] = encodeCol(pdfTrain, "FIELD_7")
printRuntime()

['CB', 'NN', 'KC', 'PV', 'na', 'HC', 'HS', 'HX', 'DN', 'NO', 'DK', 'TK', 'BT', 'DT', 'SV', 'TA', 'LS', 'XV', 'GD', 'TQ', 'TN', 'TS', 'QN', 'CK', 'XN', 'XD', 'HT', 'CN', 'TB', 'HN', 'XK', 'TC', 'CC', 'CH', 'TE', 'MS', 'HD', 'GB']
38
2020-02-12 00:16:15
-------------------


In [155]:
pdfTrain["F7_SUMTEST"] = pdfTrain["FIELD_7_ONEHOT_TEST"].apply(sum)

In [156]:
pdfTrain[["FIELD_7", "FIELD_7_VECTOR", "F7_CHECKSUM", "FIELD_7_ONEHOT", "F7_SUM", "FIELD_7_ONEHOT_TEST", "F7_SUMTEST"]].head()

,FIELD_7,FIELD_7_VECTOR,F7_CHECKSUM,FIELD_7_ONEHOT,F7_SUM,FIELD_7_ONEHOT_TEST,F7_SUMTEST
0,[],[na],1,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
1,"['GD', 'GD', 'TE']","[GD, GD, TE]",3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.0
2,"['DN', 'HN', 'DN']","[DN, HN, DN]",3,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",3.0,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",3.0
3,"['CH', 'TQ']","[CH, TQ]",2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0
4,"['SV', 'GD']","[SV, GD]",2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0


### FIELD 9

In [23]:
vocabF9, word2intF9, int2wordF9, pdfTrain["FIELD_9_ONEHOT_TEST"] = encodeCol(pdfTrain, "FIELD_9")
printRuntime()

['na', 'GD', 'DN', 'XD', 'HC', 'TN', 'CH', 'CN', 'HT', 'DT', 'XK', 'TK', 'GB', 'DK', 'SV', 'HN', 'TS', 'TA', 'HD', 'NN', 'BT', 'HS', 'HX', 'NO', 'KC', 'CB', 'TC', 'XV', '80', 'XN', 'CC', '86', '75', '79', 'MS']
35
2020-02-12 00:16:20
-------------------


In [168]:
pdfTrain[["FIELD_9", "FIELD_9_ONEHOT_TEST"]].head()

,FIELD_9,FIELD_9_ONEHOT_TEST
0,na,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,GD,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,DN,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,na,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,GD,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### FIELD 39

In [24]:
pdfTrain.loc[pdfTrain["FIELD_39"].isnull(), "FIELD_39"] = "None"

In [25]:
vocabF39, word2intF39, int2wordF39, pdfTrain["FIELD_39_ONEHOT_TEST"] = encodeCol(pdfTrain, "FIELD_39")
printRuntime()

['None', 'VN', 'JP', 'CA', 'TQ', 'CN', 'UK', 'CZ', 'KR', 'VU', '1', 'TW', 'IN', 'DL', 'HK', 'US', 'KP', 'DE', 'NL', 'NN', 'DT', 'HQ', 'N', 'SG', 'MY', 'IT', 'BE', 'TH', 'DK', 'TR', 'IL', 'TS', 'FR', 'SE', 'AU', 'AE', 'GB', 'NU', 'SC', 'PH', 'ES', 'AD', 'DM', 'TL', 'TK']
45
2020-02-12 00:16:26
-------------------


In [177]:
pdfTrain[["FIELD_39", "FIELD_39_ONEHOT_TEST"]].head()

,FIELD_39,FIELD_39_ONEHOT_TEST
0,None,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,VN,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,None,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,None,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,None,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### FIELD 13

In [26]:
vocabF13, word2intF13, int2wordF13, pdfTrain["FIELD_13_ONEHOT_TEST"] = encodeCol(pdfTrain, "FIELD_13")
printRuntime()

['None', 'BI', 'YN', 'TG', 'TB', 'QW', 'AQ', 'TO', 'TI', 'TC', 'FI', 'BO', 'HI', 'TZ', 'HZ', 'HW', 'AC', 'HK', 'TL', 'BT', 'TA', 'TJ', 'AO', 'TD', 'TN', 'TH', 'HB', 'TK', 'HF', 'HJ', 'KC', 'DA', 'TE', 'TM', 'NT', 'TX', 'FF', 'AI', 'HD', 'TP', 'HG', 'AR', 'QA', 'TY', 'QZ', 'TQ', 'AP', 'HH', 'T5', 'HE', 'HL', 'TU', 'TV', 'FE', 'TF', 'HA', 'HC', 'TW', 'TS', 'T9', 'QI', 'BQ', 'SN', 'QB', 'BD', 'BN', '0', 'SG', 'AB', 'HX', 'KT', 'AL', 'FJ', 'DW', 'BA', 'SP', 'YV', 'BH', 'TT', 'NI', 'FB', 'QD', 'YA', 'NK', 'SY', 'AA', 'HY', 'HR', 'NE', 'QG', 'EB', 'FA', 'SL', 'T7', 'NQ', 'BP', 'CA', 'SS', 'T3', 'NW', 'YF', 'DE', 'FV', 'AD', 'QF', 'NC', '8', 'QC', 'SI', 'HS', 'SK', 'QO', 'FD', 'FK', 'BK', 'AH', 'FL', 'NV', 'HP', 'NB', 'FH', 'FC', 'T1', 'EI', 'QE', 'DC', 'TR', 'QU', 'A6', 'FP', 'FR', 'SD', 'BB', 'QJ', 'QL', 'HQ', 'FS', 'AS', 'CD', 'HN', 'EA', 'CE', 'H3', 'EJ', 'CJ', 'DQ', 'DB', 'DZ', 'DT', 'SA', 'NO', 'HO', '4', 'NH', 'SO', 'QK', 'F4', 'SH', 'HM', 'EH', 'NU', 'AY', 'B2', 'EP', 'NG', 'BG', 'FM'

In [183]:
pdfTrain[["FIELD_13", "FIELD_13_ONEHOT_TEST"]].head()

,FIELD_13,FIELD_13_ONEHOT_TEST
0,None,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,BI,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,YN,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,None,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,BI,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [27]:
# pdX = pdfTrain[["label", "FIELD_7_ONEHOT_TEST", "FIELD_9_ONEHOT_TEST", 
#                 "FIELD_13_ONEHOT_TEST", "FIELD_39_ONEHOT_TEST"]].copy()
# pdX.to_pickle(basePath + "embedding_test.pickle")
pdX = pd.read_pickle(basePath + "embedding_test.pickle")
printRuntime()

2020-02-12 00:16:37
-------------------


In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

In [ ]:
# define the model
model = Sequential()
model.add(Embedding(vocabSize, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [191]:
def embedStrCol(pdf, col, lblCol, embedSz):
    """
    Do the embedding on col (which are already encoded as 1-hot)
    Target specified as lblCol
    Args:
        pdf: pandas dataframe
        col: column to be embedded
        lblCol: target column
        embedSz: embedding size
    Returns:
        
        W1, b1: Embedding matrix and bias factor
        embeddedCol: Series that are embedded by input
    """
    shape = pdf[col].shape
    W1 = []
    b1 = []
    # Build a neural network with 1 input layer (1-hot); 
    # 1 hidden layer (hidden_size configurable); 1 softmax layer
    
    # Train on label
    newColSeries = pd.Series(np.zeros(shape[0]))
    # Return
    return W1, b1, newColSeries

printRuntime()

2020-02-10 17:40:45
-------------------


# Sample of using keras to build the NN

In [194]:
from keras import Sequential
from keras.layers import Dense

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/quyennguyen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-194-e854b7e6d2e7>", line 1, in <module>
    from keras import Sequential
ModuleNotFoundError: No module named 'keras'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/quyennguyen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2039, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/quyennguyen/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_off

ModuleNotFoundError: No module named 'keras'

In [ ]:
# EXAMPLE
# We have 8 input features and one target variable. 2 Hidden layers. Each hidden layer will have 4 nodes.
classifier = Sequential()
# First Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal', input_dim=8))
# Second  Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
# Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
# --- #

# In case of our data, we have vocabSize features (1-hot vector), 1 hidden layer with embedSz nodes, 1 output
classifier = Sequential()
# First Hidden Layer
classifier.add(Dense(embedSz, activation='relu', kernel_initializer='random_normal', input_dim=vocabSize))
# Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [ ]:
# Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [195]:
X = pdfTrain[col].values
y = pdfTrain[lblCol].values

<a id="current">Current</a>

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Fitting the data to the training dataset
classifier.fit(X_train,y_train, batch_size=10, epochs=100)

In [ ]:
eval_model=classifier.evaluate(X_train, y_train)
eval_model

# ---------------

In [192]:
embedSz = 10
col = "FIELD_7_ONEHOT_TEST"
lblCol = "label"
W1_F7, b1_F7, pdfTrain["FIELD7_EMBED"] = embedStrCol(pdfTrain, col, lblCol, embedSz)
printRuntime()

2020-02-10 17:40:47
-------------------


In [193]:
pdfTrain[["FIELD_7_ONEHOT_TEST", "FIELD7_EMBED"]].head()

,FIELD_7_ONEHOT_TEST,FIELD7_EMBED
0,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0
2,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0.0
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0


# Check FIELD_13

In [146]:
dfTmp = pdfTrain.groupby("FIELD_13", as_index=False).agg({"id": ["count"], "label": ["sum", "mean", "std"]})
dfTmp.columns = ["_".join(x) for x in dfTmp.columns.ravel()]
dfTmp.sort_values(["id_count"], ascending=False).head(10)

,FIELD_13_,id_count,label_sum,label_mean,label_std
224,YN,4961,77,0.015521,0.123625
24,BI,4836,73,0.015095,0.121944
196,TA,1193,17,0.014250,0.118569
28,BO,1067,20,0.018744,0.135684
221,TZ,610,11,0.018033,0.133179
164,QW,336,3,0.008929,0.094209
201,TF,315,3,0.009524,0.097279
197,TB,306,3,0.009804,0.098690
200,TE,290,5,0.017241,0.130395
198,TC,282,3,0.010638,0.102774


In [ ]:
dfTmp01 = pdfTrain.groupby(["FIELD_13", "label"], as_index=False).agg({"id": ["count"]})
dfTmp01.columns = ["_".join(x) for x in dfTmp01.columns.ravel()]

In [128]:
dfTmp01.sort_values(["FIELD_13_"], ascending=False).head(10)

,FIELD_13_,label_,id_count
266,YV,0,26
265,YN,1,77
264,YN,0,4884
263,YF,0,2
262,YA,0,3
261,TZ,1,11
260,TZ,0,599
259,TY,0,99
258,TX,1,2
257,TX,0,70


In [136]:
dfTmp.sort_values(["id"]).head(10)

,id
FIELD_13,
SS,1
EH,1
BU,1
QU,1
CB,1
CC,1
NF,1
CE,1
CH,1


In [137]:
dfTmp = pdfTest.groupby("FIELD_13").agg({"id": "count"})
dfTmp.sort_values(["id"], ascending=False).head(10)

,id
FIELD_13,
YN,3293
BI,3100
TA,799
BO,769
TZ,441
TF,237
QW,197
TG,195
TC,191


In [138]:
dfTmp.sort_values(["id"]).head(10)

,id
FIELD_13,
ZA,1
EU,1
EP,1
EJ,1
EF,1
ED,1
NZ,1
F4,1
HO,1


# CHECK FIELD_39

In [ ]:
# Maybe: Phone source

In [139]:
dfTmp = pdfTrain.groupby("FIELD_39").agg({"id": "count"})
dfTmp.sort_values(["id"], ascending=False).head(10)

,id
FIELD_39,
VN,10529
TW,157
KR,97
JP,95
CN,85
TQ,46
CZ,37
1,28
HQ,27


In [140]:
dfTmp.sort_values(["id"]).head(10)

,id
FIELD_39,
VU,1
SE,1
SC,1
PH,1
TR,1
IT,1
IL,1
TK,1
ES,1


In [141]:
dfTmp = pdfTest.groupby("FIELD_39").agg({"id": "count"})
dfTmp.sort_values(["id"], ascending=False).head(10)

,id
FIELD_39,
VN,6984
None,5942
TW,92
KR,81
JP,63
CN,48
CZ,38
TQ,30
DL,19


In [142]:
dfTmp.sort_values(["id"]).head(10)

,id
FIELD_39,
IL,1
NU,1
VU,1
ES,1
DM,1
DK,1
WS,1
AO,1
AN,1


# Convert data types

In [ ]:
for c in lsFieldFt:
    df[c] = df[c].astype(np.float64)

### Add neighbor features
Neighbor features: 
    + avg+std of groupby (province, age_group, maCv)
    + avg+std of groupby (lv3_loc, age_group, maCv)
    + avg+std of groupby (province, maCv)
    

In [157]:
lsFieldFt = [c for c in df.columns 
             if "FIELD" in c 
             and c not in ["FIELD_%d"%d for d in [7, 9, 13, 39, 40]]]
pprint(lsFieldFt)

['FIELD_1',
 'FIELD_2',
 'FIELD_3',
 'FIELD_4',
 'FIELD_5',
 'FIELD_6',
 'FIELD_11',
 'FIELD_14',
 'FIELD_15',
 'FIELD_16',
 'FIELD_18',
 'FIELD_19',
 'FIELD_20',
 'FIELD_21',
 'FIELD_22',
 'FIELD_23',
 'FIELD_25',
 'FIELD_26',
 'FIELD_27',
 'FIELD_28',
 'FIELD_29',
 'FIELD_30',
 'FIELD_31',
 'FIELD_32',
 'FIELD_33',
 'FIELD_34',
 'FIELD_35',
 'FIELD_36',
 'FIELD_37',
 'FIELD_38',
 'FIELD_41',
 'FIELD_42',
 'FIELD_44',
 'FIELD_45',
 'FIELD_46',
 'FIELD_47',
 'FIELD_48',
 'FIELD_49',
 'FIELD_50',
 'FIELD_51',
 'FIELD_52',
 'FIELD_53',
 'FIELD_54',
 'FIELD_55',
 'FIELD_56',
 'FIELD_57',
 'FIELD_10_GH',
 'FIELD_10_None',
 'FIELD_10_T1',
 'FIELD_12_0',
 'FIELD_12_1',
 'FIELD_12_HT',
 'FIELD_12_None',
 'FIELD_12_TN',
 'FIELD_17_G2',
 'FIELD_17_G3',
 'FIELD_17_G4',
 'FIELD_17_G7',
 'FIELD_17_G8',
 'FIELD_17_G9',
 'FIELD_17_GX',
 'FIELD_17_None',
 'FIELD_24_K1',
 'FIELD_24_K2',
 'FIELD_24_K3',
 'FIELD_24_None',
 'FIELD_43_0',
 'FIELD_43_5',
 'FIELD_43_A',
 'FIELD_43_B',
 'FIELD_43_C',
 'FIELD

In [159]:
aggDict = {}
for f in lsFieldFt:
    aggDict[f] = ["mean", "std"]

In [147]:
# 1
lsMetaCol1 = ["province", "group_age", "maCv"]
lsMetaCol2 = ["lv3_loc", "group_age", "maCv"]
lsMetaCol3 = ["province", "maCv"]

In [161]:
dfNb01 = df.groupby(lsMetaCol1, as_index=False).agg(aggDict)
printRuntime()

2020-02-03 23:12:12
-------------------


In [162]:
dfNb02 = df.groupby(lsMetaCol2, as_index=False).agg(aggDict)
printRuntime()
dfNb03 = df.groupby(lsMetaCol3, as_index=False).agg(aggDict)
printRuntime()

2020-02-03 23:12:15
-------------------
2020-02-03 23:12:15
-------------------


In [164]:
# Save outputs: df, dfNb01, dfNb02, dfNb03
df.to_pickle(cleanedTrain, compression="bz2")
printRuntime()

2020-02-03 23:15:05
-------------------


In [165]:
dfNb01.to_pickle(nb01, compression="bz2")
dfNb02.to_pickle(nb02, compression="bz2")
dfNb03.to_pickle(nb03, compression="bz2")
printRuntime()

2020-02-03 23:15:41
-------------------


In [ ]:
# Nummeric data: 
# Replace province/lv3_loc by (mean, std, p10, p25, p50, p75, p90) value of 3, 22
# Replace maCv by (mean, std, p10, p25, p50, p75, p90) value of

### Distribution of categorical data:
- province
- province+district
- age_source1
- age_source2
- maCv

#### province + district

In [18]:
pdfTrain["lv3_loc"] = "[" + pdfTrain["province"] + "]_[" + pdfTrain["district"] + "]"
lsProvince = pdfTrain["province"].unique()
lsLoc = pdfTrain["lv3_loc"].unique()
print(len(lsProvince), len(lsLoc))
printRuntime()

66 748
2020-01-28 21:41:43
-------------------


In [19]:
lsProvince

array([nan, 'Tỉnh Đồng Nai', 'Tỉnh Tuyên Quang', 'Thành phố Hồ Chí Minh',
       'Tỉnh Bắc Giang', 'Tỉnh Thanh Hóa', 'Tỉnh Thừa Thiên Huế',
       'Tỉnh Cà Mau', 'Tỉnh Kiên Giang', 'Tỉnh Long An', 'Tỉnh Tây Ninh',
       'Thành phố Hà Nội', 'Tỉnh Bình Dương', 'Tỉnh Hòa Bình',
       'Tỉnh Bến Tre', 'Tỉnh Tiền Giang', 'Tỉnh Hà Tĩnh', 'Tỉnh Hoà Bình',
       'Tỉnh Hậu Giang', 'Tỉnh Quảng Nam', 'Tỉnh Khánh Hòa',
       'Tỉnh Phú Yên', 'Tỉnh Phú Thọ', 'Tỉnh Vĩnh Phúc', 'Tỉnh Hưng Yên',
       'Tỉnh Bình Phước', 'Thành phố Đà Nẵng', 'Tỉnh Đồng Tháp',
       'Tỉnh Bình Định', 'Tỉnh Bà Rịa - Vũng Tàu', 'Tỉnh Quảng Ninh',
       'Tỉnh Trà Vinh', 'Tỉnh Nghệ An', 'Tỉnh Bắc Kạn', 'Tỉnh Bình Thuận',
       'Tỉnh An Giang', 'Tỉnh Thái Bình', 'Tỉnh Bạc Liêu',
       'Tỉnh Ninh Bình', 'Thành phố Hải Phòng', 'Tỉnh Quảng Ngãi',
       'Tỉnh Lâm Đồng', 'Tỉnh Gia Lai', 'Tỉnh Nam Định', 'Tỉnh Lạng Sơn',
       'Tỉnh Sơn La', 'Tỉnh Vĩnh Long', 'Tỉnh Thái Nguyên',
       'Thành phố Cần Thơ', 'Tỉnh Quảng Bìn

In [39]:
# Distribution over lv3_loc
pdfTrain.groupby(["lv3_loc", "label"]).agg({"id":("count")})
# pdfTrain[["lv3_loc", "province", "id", "label"]].hist(bins=10)

id
lv3_loc                                label     
[Thành phố Cần Thơ]_[Huyện Cờ Đỏ]      0       22
[Thành phố Cần Thơ]_[Huyện Phong Điền] 0       11
                                       1        1
[Thành phố Cần Thơ]_[Huyện Thới Lai]   0       21
[Thành phố Cần Thơ]_[Huyện Vĩnh Thạnh] 0       19
[Thành phố Cần Thơ]_[Quận Bình Thuỷ]   0       14
[Thành phố Cần Thơ]_[Quận Bình Thủy]   0        8
[Thành phố Cần Thơ]_[Quận Cái Răng]    0       17
[Thành phố Cần Thơ]_[Quận Ninh Kiều]   0       34
                                       1        1
[Thành phố Cần Thơ]_[Quận Thốt Nốt]    0       24
[Thành phố Cần Thơ]_[Quận Ô Môn]       0       20
[Thành phố Hà Nội]_[Huyện Ba Vì]       0       30
[Thành phố Hà Nội]_[Huyện Chương Mỹ]   0       44
[Thành phố Hà Nội]_[Huyện Gia Lâm]     0       36
[Thành phố Hà Nội]_[Huyện Hoài Đức]    0       24
[Thành phố Hà Nội]_[Huyện Mê Linh]     0       22
                                       1        1
[Thành phố Hà Nội]_[Huyện Mỹ Đức]      0       15
[Thành phố Hà Nội]_[Huyện Phú Xuyên]   0       18
                                       1        2
[Thành phố Hà Nội]_[Huyện Phúc Thọ]    0       14
[Thành phố Hà Nội]_[Huyện Quốc Oai]    0       28
[Thành phố Hà Nội]_[Huyện Sóc Sơn]     0       34
[Thành phố Hà Nội]_[Huyện Thanh Oai]   0       22
                                       1        1
[Thành phố Hà Nội]_[Huyện Thanh Trì]   0       22
                                       1        3
[Thành phố Hà Nội]_[Huyện Thường Tín]  0       17
[Thành phố Hà Nội]_[Huyện Thạch Thất]  0       21
...                                           ...
[Tỉnh Đồng Nai]_[Huyện Nhơn Trạch]     1        2
[Tỉnh Đồng Nai]_[Huyện Thạnh Phú]      0        1
[Tỉnh Đồng Nai]_[Huyện Thống Nhất]     0       40
[Tỉnh Đồng Nai]_[Huyện Trảng Bom]      0       96
                                       1        2
[Tỉnh Đồng Nai]_[Huyện Tân Phú]        0       59
[Tỉnh Đồng Nai]_[Huyện Vĩnh Cửu]       0       44
                                       1        1
[Tỉnh Đồng Nai]_[Huyện Xuân Lộc]       0       82
                                       1        1
[Tỉnh Đồng Nai]_[Huyện Định Quán]      0       70
[Tỉnh Đồng Nai]_[Thành phố Biên Hòa]   0      321
                                       1        1
[Tỉnh Đồng Nai]_[Thị xã Long Khánh]    0       59
[Tỉnh Đồng Tháp]_[Huyện Cao Lãnh]      0       25
[Tỉnh Đồng Tháp]_[Huyện Châu Thành]    0       37
                                       1        2
[Tỉnh Đồng Tháp]_[Huyện Hồng Ngự]      0       33
[Tỉnh Đồng Tháp]_[Huyện Lai Vung]      0       28
[Tỉnh Đồng Tháp]_[Huyện Lấp Vò]        0       41
                                       1        2
[Tỉnh Đồng Tháp]_[Huyện Tam Nông]      0       17
[Tỉnh Đồng Tháp]_[Huyện Thanh Bình]    0       34
[Tỉnh Đồng Tháp]_[Huyện Tháp Mười]     0       42
                                       1        1
[Tỉnh Đồng Tháp]_[Huyện Tân Hồng]      0       21
                                       1        1
[Tỉnh Đồng Tháp]_[Thành phố Cao Lãnh]  0       42
[Tỉnh Đồng Tháp]_[Thành phố Sa Đéc]    0       12
[Tỉnh Đồng Tháp]_[Thị xã Hồng Ngự]     0       19

[912 rows x 1 columns]

In [41]:
# Distribution over province
pdfTrain.groupby(["province", "label"], as_index=False).agg({"id":("count")})
# pdfTrain[["lv3_loc", "province", "id", "label"]].hist(bins=10)

id
province               label      
Thành phố Cần Thơ      0       190
                       1         2
Thành phố Hà Nội       0       657
                       1        10
Thành phố Hải Phòng    0       252
Thành phố Hồ Chí Minh  0      2353
                       1        36
Thành phố Đà Nẵng      0       226
                       1         2
Tỉnh An Giang          0       597
                       1        11
Tỉnh Bà Rịa - Vũng Tàu 0       300
                       1         6
Tỉnh Bình Dương        0       798
                       1         9
Tỉnh Bình Phước        0       173
                       1         4
Tỉnh Bình Thuận        0       282
                       1         5
Tỉnh Bình Định         0       308
                       1         4
Tỉnh Bạc Liêu          0       227
                       1        11
Tỉnh Bắc Giang         0       206
                       1         6
Tỉnh Bắc Kạn           0        38
Tỉnh Bắc Ninh          0       129
                       1         1
Tỉnh Bến Tre           0       329
                       1         4
...                            ...
Tỉnh Sơn La            0        76
                       1         1
Tỉnh Thanh Hóa         0       538
Tỉnh Thái Bình         0       141
Tỉnh Thái Nguyên       0       141
                       1         1
Tỉnh Thừa Thiên Huế    0       198
                       1         1
Tỉnh Tiền Giang        0       632
                       1         9
Tỉnh Trà Vinh          0       276
                       1         7
Tỉnh Tuyên Quang       0        83
Tỉnh Tây Ninh          0       327
                       1         3
Tỉnh Vĩnh Long         0       305
                       1         4
Tỉnh Vĩnh Phúc         0       105
Tỉnh Vĩnh phúc         0        17
Tỉnh Yên Bái           0        72
Tỉnh Điện Biên         0        43
                       1         2
Tỉnh Đắk Lắk           0       288
                       1         1
Tỉnh Đắk Nông          0        55
                       1         3
Tỉnh Đồng Nai          0      1023
                       1        10
Tỉnh Đồng Tháp         0       352
                       1         6

[113 rows x 1 columns]

In [133]:
dfProvince = df.groupby(["province", "label"], as_index=False).agg({"id":("count")}).sort_values(["province"])
# dfProvince.columns = ["_".join(x) for x in dfProvince.columns.ravel()]
print(dfProvince.shape)
display(dfProvince)

(112, 3)


,province,label,id
0,Thành phố Cần Thơ,0,189
1,Thành phố Cần Thơ,1,2
2,Thành phố Hà Nội,0,657
3,Thành phố Hà Nội,1,10
4,Thành phố Hải Phòng,0,252
5,Thành phố Hồ Chí Minh,0,2339
6,Thành phố Hồ Chí Minh,1,35
7,Thành phố Đà Nẵng,0,226
8,Thành phố Đà Nẵng,1,2
9,Tỉnh An Giang,0,597


In [138]:
dfProvince.pivot(index="province", columns="label", values="id")[34:]

label,0,1
province,,
Tỉnh Lạng Sơn,72.0,1.0
Tỉnh Nam Định,145.0,3.0
Tỉnh Nghệ An,411.0,3.0
Tỉnh Ninh Bình,125.0,3.0
Tỉnh Ninh Thuận,108.0,3.0
Tỉnh Phú Thọ,153.0,5.0
Tỉnh Phú Yên,156.0,4.0
Tỉnh Quảng Bình,101.0,1.0
Tỉnh Quảng Nam,372.0,3.0


#### Age

In [21]:
# Age
pdfTrain[lsCatCol].describe()

,age_source1,age_source2
count,17189.000000,20322.000000
mean,33.041015,32.872011
std,9.187672,9.096176
min,0.000000,-1.000000
25%,26.000000,26.000000
50%,31.000000,31.000000
75%,39.000000,38.000000
max,71.000000,89.000000


In [22]:
# Age source < 18: 
pdfTrainLt18 = pdfTrain[(pdfTrain["age_source1"] < 18) | (pdfTrain["age_source2"] < 18)]
print(pdfTrainLt18.shape)
pdfTrainLt18.describe()
# => Filter out <18 value for both age_source: only 76 value would not affect

(76, 65)


,id,label,age_source1,age_source2,FIELD_1,FIELD_2,FIELD_3,FIELD_4,FIELD_5,FIELD_6,...,FIELD_34,FIELD_46,FIELD_50,FIELD_51,FIELD_52,FIELD_53,FIELD_54,FIELD_55,FIELD_56,FIELD_57
count,76.000000,76.000000,68.000000,69.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,...,76.000000,76.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000
mean,15467.657895,0.026316,9.073529,27.304348,0.763158,0.723684,1960.052632,1.644737,2.368421,0.184211,...,0.592105,0.671053,33.052261,31.674101,32.816464,33.035159,0.019710,0.097681,0.003043,0.003333
std,8855.343534,0.161136,12.272182,10.871312,0.427970,0.450146,1567.825321,1.251175,1.742352,0.481955,...,0.494709,0.472953,9.858006,21.009283,7.559901,7.570787,0.070083,0.096622,0.012403,0.013897
min,213.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,29.770000,4.413000,30.955000,31.171000,0.000000,0.000000,0.000000,0.000000
25%,8847.250000,0.000000,2.750000,17.000000,1.000000,0.000000,-1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,29.770000,16.799000,30.955000,31.171000,0.000000,0.040000,0.000000,0.000000
50%,14407.500000,0.000000,4.000000,30.000000,1.000000,1.000000,1820.000000,1.000000,2.000000,0.000000,...,1.000000,1.000000,29.770000,21.478000,30.955000,31.171000,0.000000,0.050000,0.000000,0.000000
75%,22138.000000,0.000000,9.000000,34.000000,1.000000,1.000000,3365.500000,2.000000,3.000000,0.000000,...,1.000000,1.000000,29.770000,54.937000,30.955000,31.171000,0.000000,0.160000,0.000000,0.000000
max,29965.000000,1.000000,48.000000,45.000000,1.000000,1.000000,4375.000000,5.000000,7.000000,2.000000,...,1.000000,1.000000,65.068000,67.366000,63.922000,64.214000,0.380000,0.550000,0.060000,0.080000


In [24]:
pdfTrain[pdfTrain["age_source1"].isnull()].head()

,id,label,province,district,age_source1,age_source2,maCv,FIELD_1,FIELD_2,FIELD_3,...,FIELD_49,FIELD_50,FIELD_51,FIELD_52,FIELD_53,FIELD_54,FIELD_55,FIELD_56,FIELD_57,lv3_loc
0,0,0,NaN,NaN,NaN,NaN,NaN,1,1.0,2547.0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,0,NaN,NaN,NaN,NaN,NaN,1,1.0,1812.0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,0,NaN,NaN,NaN,28.0,2983,1,0.0,-1.0,...,True,29.77,4.413,30.955,31.171,0.0,0.0,0.0,0.0,NaN
8,8,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,0,NaN,NaN,NaN,NaN,NaN,1,1.0,2544.0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
pdfTrain["avg_age"] = pdfTrain.apply(lambda row: avgAge(row), axis=1)
pdfTrain["diff_age"] = pdfTrain.apply(lambda row: diffAge(row), axis=1)
printRuntime()

In [27]:
display(pdfTrain[["age_source1", "age_source2", "avg_age", "diff_age"]].head())
printRuntime()

,age_source1,age_source2,avg_age,diff_age
0,NaN,NaN,NaN,NaN
1,44.0,44.0,44.0,44.0
2,30.0,30.0,30.0,30.0
3,43.0,NaN,43.0,43.0
4,21.0,21.0,21.0,21.0


In [29]:
display(pdfTrain[["age_source1", "age_source2", "avg_age", "diff_age"]][pdfTrain["diff_age"]<0].head())
printRuntime()

,age_source1,age_source2,avg_age,diff_age
6,NaN,28.0,28.0,-28.0
12,NaN,36.0,36.0,-36.0
18,NaN,30.0,30.0,-30.0
31,NaN,43.0,43.0,-43.0
38,NaN,23.0,23.0,-23.0


2020-01-28 22:16:53
-------------------


#### maCv

### Break down maCv to (maCv[:2] + maCv[2:]) DONE!

In [15]:
pdfTrain[lsMetaCol].head()

KeyError: "['lv3_loc', 'avg_age', 'diff_age', 'group_age'] not in index"

In [62]:
def formatJob(iStr):
    if "none" == iStr or "nan" == iStr:
        return "none"
    # Replace all digit
    iStr = re.sub("\d", "", iStr)
    # Replace cn to cong nhan
    iStr = re.sub(r"^(cnhân|cn|coõng nhaõn)", "công nhân ", iStr)
    # Replace nv/nv. to nhan vien
    iStr = re.sub(r"(- nv|nv.|nv)", "nhân viên ", iStr)
    # Replace p. to phó
    iStr = re.sub(r"(^p\.)", "phó ", iStr)
    
    iStr = (iStr.replace("-", " ")
            .replace("cty", "công ty")
            .replace(".", " ")
            .replace("(", " ").
            replace(")", " ")
           )
    return iStr


def splitJobType(iStr):
    if "none" == iStr or "nan" == iStr:
        return "none", "none"
    iStr = formatJob(iStr)
    
    lsWord = iStr.split()
    if len(lsWord) == 0:
        return "none", "none"
    splitIdx = 2
    if "phó chánh" in iStr or ("phó trưởng" in iStr 
                               and "phó trưởng phòng" not in iStr
                               and "phó trưởng ban" not in iStr):
        splitIdx = 4
    elif ("phó phòng" not in iStr) and (lsWord[0] == "phó" or lsWord[0] == "trưởng"):
        splitIdx = 3
    
    jobCat = " ".join(lsWord[:splitIdx])
    jobDesc = "none"
    if len(lsWord) > splitIdx:
        jobDesc = " ".join(lsWord[splitIdx:])
    return jobCat, jobDesc

# Test
print(splitJobType("nhân viên phòng thí nghiệm"))
print(splitJobType("nhân viên bảo trì"))
print(splitJobType("cn ủi"))
print(splitJobType("9782cấp dưỡng"))
print(splitJobType("trưởng dây chuyền phòng sản xuất"))
print(splitJobType("p. trưởng phòng"))
printRuntime()

('nhân viên', 'phòng thí nghiệm')
('nhân viên', 'bảo trì')
('công nhân', 'ủi')
('cấp dưỡng', 'none')
('trưởng dây chuyền', 'phòng sản xuất')
('phó trưởng phòng', 'none')
2020-02-15 00:38:37
-------------------


In [81]:
# pdfTrain["jobCat"], pdfTrain["jobDesc"] = pdfTrain["maCv"].apply(lambda x: splitJobType(x))
%time
pdfTrain["maCv"] = pdfTrain["maCv"].apply(formatJob)
pdfTrain["maCv_vector"] = pdfTrain["maCv"].apply(lambda x: x.split())
pdfTrain["jobCat"], pdfTrain["jobDesc"] = zip(*pdfTrain["maCv"].apply(splitJobType))
printRuntime()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs
2020-02-15 21:23:47
-------------------


In [79]:
pdfTrain[["maCv", "maCv_vector", "jobCat", "jobDesc"]].head()

,maCv,maCv_vector,jobCat,jobDesc
0,none,[none],none,none
1,none,[none],none,none
2,công nhân,"[công, nhân]",công nhân,none
3,none,[none],none,none
4,none,[none],none,none


In [66]:
lsJobCat = pdfTrain["jobCat"].unique()
print(len(lsJobCat))
pprint(lsJobCat[:10])

396
array(['none', 'công nhân', 'cấp dưỡng', 'nhân viên', 'điều dưỡng',
       'y sĩ', 'giáo viên', 'kế toán', 'vscn', 'đứng máy'], dtype=object)


In [67]:
pdfTmpJobCat = pdfTrain.groupby(["jobCat"], as_index=False).agg({"id": "count"})

In [68]:
pdfTmpJobCat.sort_values(["id"], ascending=False).head(30) # replace cn => cong nhan

,jobCat,id
196,none,22896
60,công nhân,3731
190,nhân viên,1602
101,giáo viên,385
54,cán bộ,64
152,kế toán,62
157,kỹ thuật,60
41,chuyên viên,57
164,lái xe,56
156,kỹ sư,43


In [70]:
pdfTmpJobCat.sort_values(["id"]).head(20)

,jobCat,id
197,nt cắt,1
240,qc point,1
239,qc,1
234,phụ trách,1
232,phụ lái,1
230,phụ cắt,1
227,phường đội,1
225,phó xnk,1
224,phó trại,1
222,phó trưởng công an,1


In [31]:
lsJob = pdfTrain["maCv"].unique()
print(len(lsJob))
pprint(lsJob)
printRuntime()

3067
array([nan, 'None', 'Công nhân', ..., 'NV. MR', 'Thợ ép đế', 'CN  May CN'],
      dtype=object)
2020-01-28 22:19:22
-------------------


In [34]:
lsJob = pdfTrain["maCv"].unique()

In [53]:
pdfTrain[pdfTrain["maCv"]!="None"].shape

(7118, 67)

In [52]:
pdfTrain[pdfTrain["maCv"]!="None"].head(10)

,id,label,province,district,age_source1,age_source2,maCv,FIELD_1,FIELD_2,FIELD_3,...,FIELD_51,FIELD_52,FIELD_53,FIELD_54,FIELD_55,FIELD_56,FIELD_57,lv3_loc,avg_age,diff_age
2,2,0,Tỉnh Đồng Nai,Huyện Long Thành,30.0,30.0,Công nhân,0,1.0,3273.0,...,56.512,30.955,31.171,0.0,0.16,0.0,0.0,[Tỉnh Đồng Nai]_[Huyện Long Thành],30.0,30.0
6,6,0,NaN,NaN,NaN,28.0,2983,1,0.0,-1.0,...,4.413,30.955,31.171,0.0,0.00,0.0,0.0,NaN,28.0,-28.0
7,7,0,Tỉnh Bắc Giang,Huyện Yên Dũng,40.0,32.0,Công nhân,1,1.0,2906.0,...,40.683,30.955,31.171,0.0,0.13,0.0,0.0,[Tỉnh Bắc Giang]_[Huyện Yên Dũng],36.0,36.0
12,12,0,NaN,NaN,NaN,36.0,Cấp dưỡng,1,0.0,-1.0,...,65.072,30.955,31.171,0.0,0.25,0.0,0.0,NaN,36.0,-36.0
14,14,0,Tỉnh Thừa Thiên Huế,Huyện Phong Điền,21.0,21.0,Nhân viên bảo trì,1,1.0,1436.0,...,63.351,30.955,31.171,0.0,0.21,0.0,0.0,[Tỉnh Thừa Thiên Huế]_[Huyện Phong Điền],21.0,21.0
15,15,0,Tỉnh Cà Mau,Huyện Đầm Dơi,20.0,20.0,Công nhân ủi,1,1.0,721.0,...,44.561,30.955,31.171,0.0,0.14,0.0,0.0,[Tỉnh Cà Mau]_[Huyện Đầm Dơi],20.0,20.0
21,21,0,Tỉnh Tây Ninh,Thành phố Tây Ninh,29.0,29.0,Nhân viên,0,0.0,-1.0,...,45.303,30.955,31.171,0.0,0.14,0.0,0.0,[Tỉnh Tây Ninh]_[Thành phố Tây Ninh],29.0,29.0
22,22,0,Thành phố Hà Nội,Quận Thanh Xuân,38.0,38.0,Nhân viên kinh doanh,1,1.0,339.0,...,45.930,30.955,31.171,0.0,0.14,0.0,0.0,[Thành phố Hà Nội]_[Quận Thanh Xuân],38.0,38.0
25,25,0,Tỉnh Bình Dương,Thị xã Tân Uyên,20.0,20.0,Công nhân,1,1.0,1450.0,...,46.564,30.955,31.171,0.0,0.14,0.0,0.0,[Tỉnh Bình Dương]_[Thị xã Tân Uyên],20.0,20.0
29,29,0,Tỉnh Thừa Thiên Huế,Thị xã Hương Thủy,26.0,26.0,Công nhân may công nghiệp,1,1.0,3267.0,...,54.380,30.955,31.171,0.0,0.16,0.0,0.0,[Tỉnh Thừa Thiên Huế]_[Thị xã Hương Thủy],26.0,26.0


In [36]:
lsJob[:10]

array(['None', 'Công nhân', '2983', 'Cấp dưỡng', 'Nhân viên bảo trì',
       'Công nhân ủi', 'Nhân viên', 'Nhân viên kinh doanh',
       'Công nhân may công nghiệp', 'Công nhân se sợi'], dtype=object)

In [49]:
pdfTrainJob = pdfTrain.groupby(["maCv"]).agg({"id":("count")})
# pdfTrainJob.columns = ["_".join(x) for x in pdfTrainJob.columns.ravel()]

In [ ]:
Calculate bad-rate percentage by location (province + district; province)

In [71]:
docs = pdfTrain["maCv"]
words = []
for d in docs:
    words.extend(d.split())
distinctWord = list(set(words))

occurenceCount = Counter(words) 
lsCount = occurenceCount.most_common()
print(len(words), len(distinctWord))
printRuntime()

50191 1558
2020-02-15 00:43:36
-------------------


In [75]:
def countWord(iStr):
    return len(iStr.split())

In [76]:
pdfTrain["maCv_len"] = pdfTrain["maCv"].apply(countWord)

In [77]:
max(pdfTrain["maCv_len"])

42

In [ ]:
vocabSize = 1560
maxLength = 42

In [74]:
lsCount[:20]

[('none', 22882),
 ('nhân', 5371),
 ('công', 4129),
 ('viên', 2263),
 ('may', 683),
 ('sản', 406),
 ('giáo', 396),
 ('xuất', 348),
 ('máy', 299),
 ('nghiệp', 291),
 ('hành', 258),
 ('hàng', 234),
 ('kỹ', 231),
 ('vận', 218),
 ('thuật', 194),
 ('bán', 165),
 ('điện', 153),
 ('phòng', 149),
 ('bộ', 128),
 ('bảo', 127)]

In [73]:
lsSingleWords = [w for w in lsCount if w[1]==1]
print(len(lsSingleWords))
printRuntime()

721
2020-02-15 00:44:51
-------------------


# Test

In [6]:
pdfTest = pd.read_csv(testPath)

/Users/quyennguyen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 2. EDA

In [ ]:
# Compare the distribution between train & test

# 3. Simple model

### 3.1 Build features

In [166]:
df.columns

Index(['id', 'label', 'province', 'district', 'age_source1', 'age_source2',
       'maCv', 'FIELD_1', 'FIELD_2', 'FIELD_3', 'FIELD_4', 'FIELD_5',
       'FIELD_6', 'FIELD_7', 'FIELD_9', 'FIELD_11', 'FIELD_13', 'FIELD_14',
       'FIELD_15', 'FIELD_16', 'FIELD_18', 'FIELD_19', 'FIELD_20', 'FIELD_21',
       'FIELD_22', 'FIELD_23', 'FIELD_25', 'FIELD_26', 'FIELD_27', 'FIELD_28',
       'FIELD_29', 'FIELD_30', 'FIELD_31', 'FIELD_32', 'FIELD_33', 'FIELD_34',
       'FIELD_35', 'FIELD_36', 'FIELD_37', 'FIELD_38', 'FIELD_39', 'FIELD_40',
       'FIELD_41', 'FIELD_42', 'FIELD_44', 'FIELD_45', 'FIELD_46', 'FIELD_47',
       'FIELD_48', 'FIELD_49', 'FIELD_50', 'FIELD_51', 'FIELD_52', 'FIELD_53',
       'FIELD_54', 'FIELD_55', 'FIELD_56', 'FIELD_57', 'FIELD_10_GH',
       'FIELD_10_None', 'FIELD_10_T1', 'FIELD_12_0', 'FIELD_12_1',
       'FIELD_12_HT', 'FIELD_12_None', 'FIELD_12_TN', 'FIELD_17_G2',
       'FIELD_17_G3', 'FIELD_17_G4', 'FIELD_17_G7', 'FIELD_17_G8',
       'FIELD_17_G9', 'FIELD_17

In [169]:
dfBk01 = dfNb01.copy()

In [ ]:
dfNb01.columns = ["_".join(x) for x in dfNb01.columns.ravel()]

In [175]:
dfNb01.columns = ["_".join(x) for x in dfNb01.columns.ravel()]
dfNb01 = dfNb01.rename(columns={"province_": "province", 
                       "group_age_": "group_age", 
                       "maCv_": "maCv"})

In [174]:
df.head()

,id,label,province,district,age_source1,age_source2,maCv,FIELD_1,FIELD_2,FIELD_3,...,FIELD_43_C,FIELD_43_D,FIELD_43_None,avg_age,diff_age,lv3_loc,FIELD_8_FEMALE,FIELD_8_MALE,FIELD_8_None,group_age
0,0,0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2547.0,...,0.0,0.0,1.0,NaN,NaN,NaN,0.0,0.0,1.0,AGE_NONE
1,1,0,Tỉnh Đồng Nai,Thành phố Biên Hòa,44.0,44.0,NaN,1.0,0.0,-1.0,...,0.0,0.0,1.0,44.0,44.0,[Tỉnh Đồng Nai]_[Thành phố Biên Hòa],0.0,1.0,0.0,AGE_III
2,2,0,Tỉnh Đồng Nai,Huyện Long Thành,30.0,30.0,Công nhân,0.0,1.0,3273.0,...,0.0,0.0,1.0,30.0,30.0,[Tỉnh Đồng Nai]_[Huyện Long Thành],1.0,0.0,0.0,AGE_II
3,3,0,Tỉnh Tuyên Quang,Thành phố Tuyên Quang,43.0,NaN,NaN,0.0,1.0,3991.0,...,0.0,0.0,1.0,43.0,43.0,[Tỉnh Tuyên Quang]_[Thành phố Tuyên Quang],0.0,0.0,1.0,AGE_III
4,4,0,Thành phố Hồ Chí Minh,Quận 1,21.0,21.0,NaN,0.0,1.0,1450.0,...,0.0,0.0,1.0,21.0,21.0,[Thành phố Hồ Chí Minh]_[Quận 1],0.0,1.0,0.0,AGE_I


In [176]:
dfNb01.head()

,province,group_age,maCv,FIELD_1_mean,FIELD_1_std,FIELD_2_mean,FIELD_2_std,FIELD_3_mean,FIELD_3_std,FIELD_4_mean,...,FIELD_43_D_mean,FIELD_43_D_std,FIELD_43_None_mean,FIELD_43_None_std,FIELD_8_FEMALE_mean,FIELD_8_FEMALE_std,FIELD_8_MALE_mean,FIELD_8_MALE_std,FIELD_8_None_mean,FIELD_8_None_std
0,Thành phố Cần Thơ,AGE_I,CN-DÁN HỢP,1.0,NaN,1.0,NaN,1451.0,NaN,0.0,...,0.0,NaN,1.0,NaN,1.0,NaN,0.0,NaN,0.0,NaN
1,Thành phố Cần Thơ,AGE_I,Công nhân,1.0,0.0,1.0,0.0,1263.5,761.554003,0.0,...,0.0,0.0,1.0,0.0,0.5,0.707107,0.5,0.707107,0.0,0.0
2,Thành phố Cần Thơ,AGE_I,Công nhân dập đầu vis - Xưởng Inox,1.0,NaN,1.0,NaN,1437.0,NaN,0.0,...,0.0,NaN,1.0,NaN,0.0,NaN,1.0,NaN,0.0,NaN
3,Thành phố Cần Thơ,AGE_II,Bôi keo,1.0,NaN,1.0,NaN,2899.0,NaN,2.0,...,0.0,NaN,1.0,NaN,1.0,NaN,0.0,NaN,0.0,NaN
4,Thành phố Cần Thơ,AGE_II,Bảo Vệ,0.0,NaN,1.0,NaN,2895.0,NaN,1.0,...,0.0,NaN,1.0,NaN,0.0,NaN,1.0,NaN,0.0,NaN


#### raw ft + nb ft + standardize raw ft

In [181]:
lsMetaCol = ["id", "label", "province", "district", "lv3_loc", 
              "age_source1", "age_source2", "avg_age", "diff_age", "group_age",
              "maCv",]
# raw ft
lsFieldFt = [c for c in df.columns 
             if "FIELD" in c 
             and c not in ["FIELD_%d"%d for d in [7, 9, 13, 39, 40]]]
label = "label"
printRuntime()

2020-02-03 23:33:00
-------------------


In [178]:
# nb ft
lsNbMetaCol = lsMetaCol1
dfNb = dfNb01 # TODO: dfNb02, dfNb03

In [186]:
# pdfDistRaw = pd.merge(pdfArea, pdfArea, on="area_level_2", how="inner")
dfITrain = pd.merge(df, dfNb01, on=lsNbMetaCol, how="left")

In [187]:
dfITrain.shape

(29924, 244)

In [188]:
dfITrain.head()

,id,label,province,district,age_source1,age_source2,maCv,FIELD_1,FIELD_2,FIELD_3,...,FIELD_43_D_mean,FIELD_43_D_std,FIELD_43_None_mean,FIELD_43_None_std,FIELD_8_FEMALE_mean,FIELD_8_FEMALE_std,FIELD_8_MALE_mean,FIELD_8_MALE_std,FIELD_8_None_mean,FIELD_8_None_std
0,0,0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2547.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,Tỉnh Đồng Nai,Thành phố Biên Hòa,44.0,44.0,NaN,1.0,0.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,Tỉnh Đồng Nai,Huyện Long Thành,30.0,30.0,Công nhân,0.0,1.0,3273.0,...,0.046154,0.211451,0.938462,0.242186,0.6,0.49371,0.4,0.49371,0.0,0.0
3,3,0,Tỉnh Tuyên Quang,Thành phố Tuyên Quang,43.0,NaN,NaN,0.0,1.0,3991.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,Thành phố Hồ Chí Minh,Quận 1,21.0,21.0,NaN,0.0,1.0,1450.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
# standardize ft
for c in lsFieldFt:
    dfITrain["%s_stdized"%c] = (dfITrain[c] - dfITrain[c+"_mean"])/dfITrain[c+"_std"]
printRuntime()

2020-02-03 23:40:06
-------------------


In [190]:
dfITrain.shape

(29924, 320)

In [191]:
iTrainPath = basePath + "itrain.pickle"
dfITrain.to_pickle(iTrainPath, compression="bz2")
printRuntime()

2020-02-03 23:40:36
-------------------


In [ ]:
# 5-fold XGBoost